In [89]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Output library versions
print(f"numpy version: {np.__version__}")
print(f"tensorflow version: {tf.__version__}")

numpy version: 1.26.4
tensorflow version: 2.18.0


In [90]:
# ----------------------------
# Constants and Encoding Legend
# ----------------------------
START_TOKEN = 13
END_TOKEN = 14

ENCODING_LEGEND = {
    'MRI_CCS_11': 1, 'MRI_EXU_95': 2, 'MRI_FRR_18': 3, 'MRI_FRR_257': 4,
    'MRI_FRR_264': 5, 'MRI_FRR_3': 6, 'MRI_FRR_34': 7, 'MRI_MPT_1005': 8,
    'MRI_MSR_100': 9, 'MRI_MSR_104': 10, 'MRI_MSR_21': 11, 'MRI_MSR_34': 12,
    'START': START_TOKEN,
    'END': END_TOKEN
}
# Build reverse mapping for decoding:
reverse_encoding = {v: k for k, v in ENCODING_LEGEND.items()}

CHAR_TO_INT = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '10': 10,
    '11': 11,
    '12': 12,
}

In [91]:
# ----------------------------
# Data Loading and Sequence Splitting
# ----------------------------
data_file = "encoded_182625.csv"
data = pd.read_csv(data_file)
print("Loaded CSV with columns:", data.columns.tolist())

# Split sequences based on "SeqOrder" (reset to 0 indicates new sequence)
sequences_tokens = []
sequences_times = []

current_tokens = []
current_times = []

for idx, row in data.iterrows():
    seq_order = row['SeqOrder']
    s_id = float(row['sourceID'])
    t_diff = float(row['timediff'])
    if seq_order == 0 and current_tokens:
        token_seq = [START_TOKEN] + [int(x) for x in current_tokens] + [END_TOKEN]
        time_seq = [0.0] + current_times
        if len(time_seq) < len(token_seq):
            time_seq = time_seq + [time_seq[-1]]
        sequences_tokens.append(token_seq)
        sequences_times.append(time_seq)
        current_tokens = []
        current_times = []
    current_tokens.append(s_id)
    current_times.append(t_diff)
if current_tokens:
    token_seq = [START_TOKEN] + [int(x) for x in current_tokens] + [END_TOKEN]
    time_seq = [0.0] + current_times
    if len(time_seq) < len(token_seq):
        time_seq = time_seq + [time_seq[-1]]
    sequences_tokens.append(token_seq)
    sequences_times.append(time_seq)

print(f"Found {len(sequences_tokens)} sequences.")

Loaded CSV with columns: ['SeqOrder', 'sourceID', 'timediff', 'PTAB', 'BodyGroup_from', 'BodyGroup_to']
Found 186 sequences.


In [92]:
# ----------------------------
# Prepare Training Data and Masks
# ----------------------------
# For each sequence, use input tokens = tokens[:-1] and target times = times[1:]
X_list, Y_list, masks_list = [], [], []
for tokens, times in zip(sequences_tokens, sequences_times):
    x_seq = tokens[:-1]    # input
    y_seq = times[1:]      # target cumulative times
    # Mask: valid tokens are those not equal to the pad value (we use END_TOKEN for padding)
    mask_seq = [1 if t != END_TOKEN else 0 for t in x_seq]
    X_list.append(x_seq)
    Y_list.append(y_seq)
    masks_list.append(mask_seq)

max_len = max(len(x) for x in X_list)
X_train = pad_sequences(X_list, maxlen=max_len, padding='post', value=END_TOKEN)
Y_cum_target = pad_sequences(Y_list, maxlen=max_len, padding='post', value=0.0)
mask_train = pad_sequences(masks_list, maxlen=max_len, padding='post', value=0)

X_train = np.array(X_train, dtype=np.int32)
Y_cum_target = np.array(Y_cum_target, dtype=np.float32)
mask_train = np.array(mask_train, dtype=np.float32)

X_train = X_train.reshape(1, -1)
Y_cum_target = Y_cum_target.reshape(1, -1)
mask_train = mask_train.reshape(1, -1)

print("X_train shape:", X_train.shape)
print("Y_cum_target shape:", Y_cum_target.shape)
print("Mask shape:", mask_train.shape)


X_train shape: (1, 8370)
Y_cum_target shape: (1, 8370)
Mask shape: (1, 8370)


In [93]:
# ----------------------------
# Target Computation Function
# ----------------------------
def compute_true_targets(cumulative_times):
    """
    Given cumulative times (shape: [batch, L_target]), compute:
      - true_proportions: increments normalized by total time.
      - true_total: total time per sequence.
    """
    diffs = cumulative_times[:, 0:1]
    diffs = tf.concat([diffs, cumulative_times[:, 1:] - cumulative_times[:, :-1]], axis=1)
    true_total = cumulative_times[:, -1:]
    true_total_safe = tf.where(true_total == 0, tf.ones_like(true_total), true_total)
    true_proportions = diffs / true_total_safe
    return true_proportions, true_total

true_prop, true_total = compute_true_targets(tf.convert_to_tensor(Y_cum_target))

In [94]:
# ----------------------------
# Transformer Components (unchanged)
# ----------------------------
def positional_encoding(length, depth):
    depth = depth / 2
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :] / depth
    angle_rates = 1 / (10000 ** depths)
    angle_rads = positions * angle_rates
    pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(layers.Layer):
    def __init__(self, vocab_size, d_model, max_len=16384, use_embedding=True):
        super(PositionalEmbedding, self).__init__()
        self.d_model = d_model
        self.use_embedding = use_embedding
        if self.use_embedding:
            self.embedding = layers.Embedding(vocab_size, d_model, mask_zero=True)
        else:
            self.embedding = layers.Dense(d_model, activation="relu")
        self.max_len = max_len
        self.pos_encoding = positional_encoding(self.max_len, d_model)
    
    def compute_mask(self, *args, **kwargs):
        if self.use_embedding:
            return self.embedding.compute_mask(*args, **kwargs)
        else:
            return None
    
    def call(self, x):
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        seq_len = tf.shape(x)[1]
        x += self.pos_encoding[tf.newaxis, :seq_len, :]
        return x

class FeedForward(layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            layers.Dense(dff, activation='relu'),
            layers.Dense(d_model),
            layers.Dropout(dropout_rate)
        ])
        self.add = layers.Add()
        self.layer_norm = layers.LayerNormalization()
    
    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x)
        return x

class CausalSelfAttention(layers.Layer):
    def __init__(self, num_heads, d_model, dropout_rate=0.1):
        super().__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model, dropout=dropout_rate)
        self.add = layers.Add()
        self.layer_norm = layers.LayerNormalization()
    
    def call(self, x):
        attn_output = self.mha(query=x, key=x, value=x, use_causal_mask=True)
        x = self.add([x, attn_output])
        x = self.layer_norm(x)
        return x

class SelfAttentionFeedForwardLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()
        self.self_attention = CausalSelfAttention(num_heads=num_heads, d_model=d_model, dropout_rate=dropout_rate)
        self.ffn = FeedForward(d_model, dff, dropout_rate)
    
    def call(self, x):
        x = self.self_attention(x)
        x = self.ffn(x)
        return x

class Encoder(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1, max_len=16384):
        super().__init__()
        self.pos_embedding = PositionalEmbedding(vocab_size, d_model)
        self.enc_layers = [SelfAttentionFeedForwardLayer(d_model, num_heads, dff, dropout_rate)
                           for _ in range(num_layers)]
        self.dropout = layers.Dropout(dropout_rate)
    
    def call(self, x):
        x = self.pos_embedding(x)
        x = self.dropout(x)
        for layer in self.enc_layers:
            x = layer(x)
        return x

class Decoder(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, dropout_rate=0.1, max_len=16384):
        super().__init__()
        self.pos_embedding = PositionalEmbedding(vocab_size, d_model)
        self.dropout = layers.Dropout(dropout_rate)
        self.dec_layers = [SelfAttentionFeedForwardLayer(d_model, num_heads, dff, dropout_rate)
                           for _ in range(num_layers)]
    
    def call(self, x, context):
        x = self.pos_embedding(x)
        x = self.dropout(x)
        for layer in self.dec_layers:
            x = layer(x)
        return x

In [95]:
# ----------------------------
# TimeDiffTransformer Model (Dual-Target)
# ----------------------------
class TimeDiffTransformer(tf.keras.Model):
    """
    This model predicts:
      1. A sequence of per-step proportions (via softmax, summing to 1 over valid tokens).
      2. An overall total time (nonnegative).
    Predicted increments = proportions * total time,
    and cumulative sum of increments gives predicted cumulative times.
    """
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, dropout_rate=0.1, max_len=16384):
        super().__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, dropout_rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, input_vocab_size, dropout_rate)
        self.proportion_head = layers.Dense(1)
        self.total_time_head = layers.Dense(1, activation='relu')
    
    def call(self, inputs):
        encoder_out = self.encoder(inputs)
        decoder_out = self.decoder(inputs, encoder_out)
        proportions_logits = self.proportion_head(decoder_out)  # (batch, seq_len, 1)
        proportions_logits = tf.squeeze(proportions_logits, axis=-1)  # (batch, seq_len)
        proportions = tf.nn.softmax(proportions_logits, axis=-1)
        pooled_encoder = tf.reduce_mean(encoder_out, axis=1)
        total_time = self.total_time_head(pooled_encoder)
        return proportions, total_time
    
    def predict_time_differences(self, inputs):
        proportions, total_time = self(inputs)
        pred_increments = proportions * total_time
        pred_cumulative = tf.math.cumsum(pred_increments, axis=1)
        return proportions, pred_increments, pred_cumulative

In [96]:
# ----------------------------
# Loss Functions with Masking for Proportions
# ----------------------------
def masked_proportion_loss(y_true, y_pred, mask):
    loss = tf.square(y_true - y_pred)  # now shape (batch, L)
    loss *= tf.cast(mask, tf.float32)
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

def total_time_loss(y_true, y_pred):
    return tf.keras.losses.MeanSquaredError()(y_true, y_pred)

In [97]:
# ----------------------------
# Model Instantiation, Compilation, and Training
# ----------------------------
vocab_size = max(ENCODING_LEGEND.values()) + 1
max_seq_len = X_train.shape[1]
model = TimeDiffTransformer(num_layers=3, d_model=32, num_heads=8, dff=128,
                            input_vocab_size=vocab_size, dropout_rate=0.1, max_len=max_seq_len)

_ = model(X_train)
model.summary()

c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. D

Model: "time_diff_transformer_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_9 (Encoder)             │ ?                      │       126,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_9 (Decoder)             │ ?                      │       126,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_138 (Dense)               │ (1, 8370, 1)           │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_139 (Dense)               │ (1, 1)                 │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 253,314 (989.51 KB)

 Trainable params: 253,314 (989.51 KB)

 Non-trainable params: 0 (0.00 B)

In [98]:
# For training, we'll use a custom training loop to apply the mask for the proportions loss.
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(x, y_cum, mask):
    with tf.GradientTape() as tape:
        pred_props, pred_total = model(x)
        # Compute true targets.
        true_props, true_total = compute_true_targets(y_cum)
        loss_props = masked_proportion_loss(true_props, pred_props, mask)
        loss_total = total_time_loss(true_total, pred_total)
        total_loss = loss_props + loss_total
    grads = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return total_loss, loss_props, loss_total

# Train for a few epochs.
epochs = 20
for epoch in range(epochs):
    loss_total_val, loss_props_val, loss_total_branch_val = train_step(X_train, Y_cum_target, mask_train)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss_total_val.numpy():.4f} (Props: {loss_props_val.numpy():.4f}, Total: {loss_total_branch_val.numpy():.4f})")

Epoch 1/20 - Loss: 6104.2061 (Props: 6103.8882, Total: 0.3177)
Epoch 2/20 - Loss: 6103.8823 (Props: 6103.8823, Total: 0.0000)
Epoch 3/20 - Loss: 6103.8735 (Props: 6103.8735, Total: 0.0000)
Epoch 4/20 - Loss: 6103.8584 (Props: 6103.8584, Total: 0.0000)
Epoch 5/20 - Loss: 6103.8306 (Props: 6103.8306, Total: 0.0000)
Epoch 6/20 - Loss: 6103.7617 (Props: 6103.7617, Total: 0.0000)
Epoch 7/20 - Loss: 6103.5977 (Props: 6103.5977, Total: 0.0000)
Epoch 8/20 - Loss: 6103.3677 (Props: 6103.3677, Total: 0.0000)
Epoch 9/20 - Loss: 6103.2056 (Props: 6103.2056, Total: 0.0000)
Epoch 10/20 - Loss: 6103.1455 (Props: 6103.1455, Total: 0.0000)
Epoch 11/20 - Loss: 6103.1221 (Props: 6103.1221, Total: 0.0000)
Epoch 12/20 - Loss: 6103.1123 (Props: 6103.1123, Total: 0.0000)
Epoch 13/20 - Loss: 6103.1064 (Props: 6103.1064, Total: 0.0000)
Epoch 14/20 - Loss: 6103.1021 (Props: 6103.1021, Total: 0.0000)
Epoch 15/20 - Loss: 6103.0996 (Props: 6103.0996, Total: 0.0000)
Epoch 16/20 - Loss: 6103.0981 (Props: 6103.0981, 

In [102]:
# ----------------------------
# Inference and CSV Output (Fully Fixed for Individual Sequences)
# ----------------------------

# Get predictions for the entire batch
proportions_pred, increments_pred, cumulative_pred = model.predict_time_differences(X_train)

# Convert ground truth arrays to NumPy for convenience
gt_increments = np.concatenate([Y_cum_target[:, 0:1], Y_cum_target[:, 1:] - Y_cum_target[:, :-1]], axis=1)
gt_cumulative = Y_cum_target

all_outputs = []

# Loop over each sequence in the batch
for seq_idx in range(X_train.shape[0]):
    # Get predictions and corresponding mask for the sequence
    props = proportions_pred[seq_idx]  # shape: (L,)
    incs = increments_pred[seq_idx]  # shape: (L,)
    cumuls = cumulative_pred[seq_idx]  # shape: (L,)
    gt_incs = gt_increments[seq_idx]  # shape: (L,)
    gt_cumuls = gt_cumulative[seq_idx]  # shape: (L,)

    # Retrieve valid indices (not padding) from the mask
    valid_indices = np.where(mask_train[seq_idx] == 1)[0]

    # Ensure that valid_indices is properly converted to a TensorFlow tensor
    valid_indices_tf = tf.convert_to_tensor(valid_indices, dtype=tf.int32)

    # Initialize step tracker
    filtered_indices = []
    decoded_sourceIDs = []
    step_counter = 1  # Reset step count for each sequence

    # Identify individual sequences using SeqOrder resets (new sequence starts at 0)
    for idx in valid_indices:
        if idx >= X_train.shape[1]:  # Prevent indexing out of bounds
            continue
        token = int(X_train[seq_idx, idx])  # Ensure int32 conversion

        if token == START_TOKEN:
            continue  # Skip start token

        filtered_indices.append(idx)

        if token == END_TOKEN:
            decoded_sourceIDs.append("END")
        else:
            decoded_sourceIDs.append(reverse_encoding.get(token, f"UNK({token})"))

    # Ensure filtered indices are valid
    if len(filtered_indices) == 0:
        continue  # Skip this sequence if no valid tokens remain

    # Convert filtered indices to TensorFlow tensor
    filtered_indices_tf = tf.convert_to_tensor(filtered_indices, dtype=tf.int32)

    # Use tf.gather() to safely extract valid prediction values
    pred_props_filtered = tf.gather(props, filtered_indices_tf).numpy()
    pred_incs_filtered = tf.gather(incs, filtered_indices_tf).numpy()
    pred_cumuls_filtered = tf.gather(cumuls, filtered_indices_tf).numpy()
    gt_incs_filtered = tf.gather(gt_incs, filtered_indices_tf).numpy()
    gt_cumuls_filtered = tf.gather(gt_cumuls, filtered_indices_tf).numpy()

    # Create DataFrame for this sequence (reset step numbering for each sequence)
    df = pd.DataFrame({
        "Sequence": seq_idx,  # Sequence index
        "Step": np.arange(1, len(decoded_sourceIDs) + 1),  # Step within this sequence
        "SourceID": decoded_sourceIDs,
        "Predicted_Proportion": pred_props_filtered,
        "Predicted_Increment": pred_incs_filtered,
        "Predicted_Cumulative": pred_cumuls_filtered,
        "GroundTruth_Increment": gt_incs_filtered,
        "GroundTruth_Cumulative": gt_cumuls_filtered
    })

    all_outputs.append(df)

# Concatenate all sequences into one DataFrame and save as CSV
if all_outputs:
    output_df = pd.concat(all_outputs, ignore_index=True)
    output_csv = "predictions_SplitModel_all_sequences.csv"
    output_df.to_csv(output_csv, index=False)
    print(f"\n✅ Predictions saved to {output_csv}")
    print("\n📌 Sample predictions:")
    print(output_df.head(10))
else:
    print("\n⚠️ No valid sequences were found for output. Check data processing.")


c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. D


✅ Predictions saved to predictions_SplitModel_all_sequences.csv

📌 Sample predictions:
   Sequence  Step     SourceID  Predicted_Proportion  Predicted_Increment   
0         0     1  MRI_MSR_104              0.000001                  0.0  \
1         0     2       UNK(0)              0.000001                  0.0   
2         0     3  MRI_FRR_257              0.000001                  0.0   
3         0     4  MRI_FRR_264              0.000002                  0.0   
4         0     5  MRI_FRR_264              0.000002                  0.0   
5         0     6   MRI_CCS_11              0.000002                  0.0   
6         0     7   MRI_CCS_11              0.000002                  0.0   
7         0     8  MRI_FRR_257              0.000001                  0.0   
8         0     9  MRI_FRR_264              0.000002                  0.0   
9         0    10  MRI_FRR_264              0.000002                  0.0   

   Predicted_Cumulative  GroundTruth_Increment  GroundTruth_Cumu

c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'causal_self_attention_59' (of type CausalSelfAttention) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'sequential_59' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\lukis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'feed_forward_59' (of type FeedForward) was passed an input with a mask attached to it. However, this layer does not support masking and 